In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import itertools
from tqdm import tqdm

In [3]:
df = pd.read_csv('logP.csv')
df

,Unnamed: 0,smiles,logP,QED_score,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0,COc1ccccc1[C@@H]1CC(=O)Nc2cc(C)c(C)cc21,4.17,0.909,12.053455,12.053455,0.045556,0.045556,0.909253,281.355,...,0,0,0,0,0,0,0,0,0,0
1,1,COc1ccc2c(c1)O[C@](O)(C(F)(F)F)CC2=O,2.79,0.839,12.616874,12.616874,0.009514,-5.052156,0.839542,262.183,...,0,0,0,0,0,0,0,0,0,0
2,2,C=C(c1ccc(N(C)C)cc1)c1ccc(N(C)C)cc1,5.30,0.828,4.217020,4.217020,1.056122,1.056122,0.828333,266.388,...,0,0,0,0,0,0,0,0,0,0
3,3,Cc1ccc([C@@H](O)C#CC(O)(c2ccccc2)c2ccccc2)o1,3.53,0.726,11.239605,11.239605,0.364327,-1.508792,0.726080,318.372,...,0,0,0,0,0,0,0,0,0,0
4,4,CC(C)(C)c1cc(Cc2ccccc2)n[nH]1,4.35,0.816,4.352931,4.352931,0.140811,0.140811,0.816043,214.312,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,239,Cc1cccc(NC(=O)C[C@@H]2S/C(=N\c3c(C)cccc3C)N(C)...,4.75,0.863,12.562081,12.562081,0.093167,-0.455663,0.863227,381.501,...,1,0,0,0,0,0,0,0,0,0
240,240,CCOc1ccccc1OCC(=O)Nc1ccc2ccccc2c1,4.73,0.737,12.114099,12.114099,0.072506,-0.211498,0.737878,321.376,...,0,0,0,0,0,0,0,0,0,0
241,241,OCC#CC(O)(c1ccc(F)cc1)c1ccc(F)cc1,3.61,0.824,12.974979,12.974979,0.339313,-1.732014,0.824192,274.266,...,0,0,0,0,0,0,0,0,0,0
242,242,CC(=O)Nc1ccc(/N=C/c2c([O-])oc3ccccc3c2=O)cc1,3.64,0.750,12.347197,12.347197,0.125336,-0.724707,0.750035,321.312,...,0,0,0,0,0,0,0,0,0,0


In [4]:
y = df.logP
X = df.iloc[:, 3:]
feats = X.columns
X


,QED_score,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0.909,12.053455,12.053455,0.045556,0.045556,0.909253,281.355,262.203,281.141579,108,...,0,0,0,0,0,0,0,0,0,0
1,0.839,12.616874,12.616874,0.009514,-5.052156,0.839542,262.183,253.111,262.045293,98,...,0,0,0,0,0,0,0,0,0,0
2,0.828,4.217020,4.217020,1.056122,1.056122,0.828333,266.388,244.212,266.178299,104,...,0,0,0,0,0,0,0,0,0,0
3,0.726,11.239605,11.239605,0.364327,-1.508792,0.726080,318.372,300.228,318.125594,120,...,0,0,0,0,0,0,0,0,0,0
4,0.816,4.352931,4.352931,0.140811,0.140811,0.816043,214.312,196.168,214.146999,84,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,0.863,12.562081,12.562081,0.093167,-0.455663,0.863227,381.501,358.317,381.151098,140,...,1,0,0,0,0,0,0,0,0,0
240,0.737,12.114099,12.114099,0.072506,-0.211498,0.737878,321.376,302.224,321.136493,122,...,0,0,0,0,0,0,0,0,0,0
241,0.824,12.974979,12.974979,0.339313,-1.732014,0.824192,274.266,262.170,274.080536,102,...,0,0,0,0,0,0,0,0,0,0
242,0.750,12.347197,12.347197,0.125336,-0.724707,0.750035,321.312,308.208,321.088080,120,...,0,0,0,0,0,0,0,0,0,0


In [21]:
def feature_selection(X, y, features: list):

    r2_values = []

    for feature_subset in tqdm(features):

        model = linear_model.LinearRegression()

        X_train = X[feature_subset]

        model.fit(X_train, y)
        r2_value = model.score(X_train, y)

        r2_values.append((r2_value, ' '.join(feature_subset)))

    r2_df = pd.DataFrame(r2_values, columns=['r2', 'cols'])

    

    return r2_df.groupby('n').sort()


In [22]:
res = feature_selection(X, y, feats)
res

  0%|          | 0/209 [00:00<?, ?it/s]


ValueError: Expected 2D array, got 1D array instead:
array=[0.909 0.839 0.828 0.726 0.816 0.757 0.755 0.695 0.81  0.906 0.864 0.735
 0.841 0.707 0.717 0.842 0.754 0.841 0.789 0.807 0.727 0.826 0.873 0.757
 0.696 0.87  0.676 0.757 0.796 0.797 0.73  0.911 0.865 0.881 0.83  0.703
 0.79  0.722 0.69  0.834 0.805 0.767 0.727 0.844 0.719 0.725 0.774 0.903
 0.775 0.843 0.867 0.881 0.894 0.799 0.795 0.738 0.692 0.877 0.824 0.806
 0.757 0.715 0.729 0.889 0.684 0.753 0.909 0.76  0.883 0.822 0.713 0.721
 0.777 0.723 0.729 0.792 0.696 0.842 0.867 0.83  0.848 0.672 0.732 0.833
 0.781 0.677 0.732 0.706 0.736 0.843 0.796 0.826 0.709 0.731 0.716 0.764
 0.694 0.763 0.757 0.762 0.676 0.912 0.728 0.678 0.825 0.813 0.758 0.744
 0.733 0.844 0.874 0.704 0.775 0.905 0.804 0.839 0.712 0.837 0.709 0.775
 0.769 0.916 0.728 0.673 0.886 0.676 0.724 0.694 0.747 0.753 0.872 0.719
 0.896 0.716 0.692 0.798 0.84  0.848 0.755 0.793 0.752 0.888 0.81  0.916
 0.73  0.808 0.806 0.769 0.704 0.837 0.773 0.676 0.807 0.791 0.78